IMPORT libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import ast
import tensorflow as tf
from tensorflow.keras import layers, models
import os

2024-10-08 13:56:24.975550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 13:56:25.101077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 13:56:25.132419: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 13:56:25.425742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.list_physical_devices('GPU')

I0000 00:00:1728395794.684240       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728395794.809159       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728395794.809225       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
dataset = pd.read_csv("/app/AI-Module/Resources/Datasets/how2sign.csv")

In [4]:
puntos_folder = "/app/AI-Module/Resources/Datasets/Points"
files = [puntos_folder + "/" + file for file in os.listdir(puntos_folder)]
len(files)

500

In [5]:
len(dataset)

500

In [6]:
def load_points(files):
    for file in files:
        item_with_index = np.load(file, allow_pickle=True)
        # Convertir los arrays a float32 para reducir memoria
        item = item_with_index[0].astype(np.float32)
        index = item_with_index[1]
        yield item, index

puntos_list = []
ids_list = []

for item, index in load_points(files):
    puntos_list.append(item)
    ids_list.append(index)

df_puntos = pd.DataFrame({
    'points': puntos_list,
    'id': ids_list
})
df_puntos = df_puntos.sort_values(["id"]).reset_index(drop=True)

In [7]:
dataset = dataset.merge(df_puntos, on="id", how="inner")

In [8]:
dataset.head()

,points_x,translation,id,len_keyframes,points_y
0,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...",If you like to find more about my services you...,448,817,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."
1,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...",Hello welcome my name is Julio Nutt and I am a...,210,401,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."
2,"[[1.0, 0.0589, 1.0, 1.0, 0.7364, 0.0, 0.6998, ...","All right, the drink we're about to make is ca...",228,308,"[[1.0, 0.0589, 1.0, 1.0, 0.7364, 0.0, 0.6998, ..."
3,"[[1.0, 0.0515, 1.0, 1.0, 0.7142, 0.0, 0.6933, ...","Okay, another thing that for me is important. ...",187,367,"[[1.0, 0.0515, 1.0, 1.0, 0.7142, 0.0, 0.6933, ..."
4,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0...","All right, we're ready to start our fire. What...",99,453,"[[-1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0..."


In [9]:
type(dataset["points_y"].iloc[0])

numpy.ndarray

In [10]:
dataset["points"]=dataset["points_y"]

Cambio el formato de points para que sea una lista de 48x48

In [11]:
import sys
# Agrega la ruta donde está ubicado Points2VecClass.py
sys.path.append('/app/AI-Module/Modules')
from Points2VecClass import Point2Vec
def pointsToCnnInputForm(points):
    p2v = Point2Vec(4)
    return p2v.CNNMatrix(points)

In [12]:
def to_cnn_input_form(serie):
    for index, element in enumerate(serie):
        element= pointsToCnnInputForm(element)
        serie.at[index]=element
    return serie
dataset['points']=to_cnn_input_form(dataset['points'])

Tokenizo las traducciones para que sean compatibles con el modelo

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Crear el tokenizador
tokenizer = Tokenizer()

# Ajustar el tokenizador al texto de las traducciones
tokenizer.fit_on_texts(dataset['translation'])

# Convertir las oraciones en secuencias de enteros
dataset['translation'] = tokenizer.texts_to_sequences(dataset['translation'])

In [14]:
from sklearn.model_selection import train_test_split


# Divide en train y test
points_train_val, points_test, translation_train_val, translation_test = train_test_split(
    dataset[["points", "len_keyframes"]], dataset["translation"], test_size=0.2, random_state=42
)
print(f"Train_val size: {len(points_train_val)}, Test size: {len(points_test)}")
points_train, points_val, translation_train, translation_val=train_test_split(
    points_train_val,translation_train_val, test_size=0.25,random_state=42
)
print(f"Train size: {len(points_train)}, Test size: {len(points_val)}")
train = {
    "points": points_train["points"],
    "len_keyframes": points_train["len_keyframes"],
    "translation_sequence": translation_train
}

val = {
    "points": points_val["points"],
    "len_keyframes": points_val["len_keyframes"],
    "translation_sequence": translation_val
}

test = {
    "points": points_test["points"],
    "len_keyframes": points_test["len_keyframes"],
    "translation_sequence": translation_test
}


Train_val size: 400, Test size: 100
Train size: 300, Test size: 100


In [15]:
import gc

globals().pop('dataset', None)
globals().pop('puntos_folder', None)
globals().pop('puntos_list', None)
globals().pop('df_puntos', None)
gc.collect()

11

In [20]:
len(train['translation_sequence'])

300

Creo datasets de tensorflow

In [21]:
def create_tf_dataset(df):
    def generator():
        for i in range(len(df['points'])):
            points = df['points'].values[i]
            sequence = df['translation_sequence'].values[i]
            yield points, sequence

    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(None, 48, 48), dtype=tf.float32),
            tf.TensorSpec(shape=(None), dtype=tf.int32)
        )
)


In [22]:
test_dataset = create_tf_dataset(test)
val_dataset=create_tf_dataset(val)
train_dataset= create_tf_dataset(train)

I0000 00:00:1728396095.667180       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728396095.667375       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728396095.667446       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728396102.289023       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728396102.289121       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-08

In [23]:
count = train_dataset.reduce(0, lambda x, _: x + 1)
count.numpy()

300

In [24]:
print(val_dataset.element_spec)

(TensorSpec(shape=(None, 48, 48), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))


In [25]:
for element in train_dataset.take(1):
    print(element)

(<tf.Tensor: shape=(1133, 48, 48), dtype=float32, numpy=
array([[[ 1.    ,  0.6049,  1.    , ...,  0.7828,  0.3171,  1.    ],
        [ 0.308 ,  0.7701,  0.2518, ...,  0.198 ,  0.5576,  1.    ],
        [ 0.811 ,  0.109 ,  0.3641, ...,  0.7605,  0.3062,  1.    ],
        ...,
        [ 0.4812,  0.3349,  0.2868, ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]],

       [[ 1.    ,  0.5889,  1.    , ...,  0.7625,  0.3176,  1.    ],
        [ 0.0208,  0.7787,  0.2834, ...,  0.1822,  0.5087,  1.    ],
        [ 0.7561,  0.1014,  0.2969, ...,  0.7796,  0.3118,  1.    ],
        ...,
        [ 0.4853,  0.3326,  0.2908, ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]],

       [[ 1.    ,  0.5205,  1.    , ...,  0.7323,  0.2591,  1.    ],
        [ 0.0196

2024-10-08 14:01:51.260875: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
def print_lenForBucketing(ds,limites):

    # Crear un DataFrame vacío para almacenar los resultados
    resultados = []

    # Recorrer los pares consecutivos de la lista de límites
    for lower, upper in zip(limites[:-1], limites[1:]):
        # Contar cuántos valores están entre 'lower' y 'upper'
        count = ((ds['len_keyframes'] > lower) & (ds['len_keyframes'] <= upper)).sum()
        
        # Guardar el resultado en una lista
        resultados.append((lower, upper, count))

    # Mostrar los resultados
    for lower, upper, count in resultados:
        print(f"Valores entre {lower} y {upper}: {count}")


limites = [0,300,500,700, 1000]
print(f'Dataset: test')
print_lenForBucketing(test,limites)
print(f'Dataset: train')
print_lenForBucketing(train,limites)
print(f'Dataset: val')

print_lenForBucketing(val,limites)

In [ ]:
train_dataset

In [26]:
def length_fn(points, translations):
    # La longitud será la cantidad de keyframes (primera dimensión de los puntos)
    longitud=tf.shape(points)[0]
    return longitud


# limites = [300,500,700,1100]
    
# bucket_batch_sizes = [32, 32, 16, 8,8]

def bucketed_dataset(dataset):
    limites = [300,500,700,1100]
    
    bucket_batch_sizes = [16, 16, 16, 8, 4]
    return dataset.bucket_by_sequence_length(
            element_length_func=length_fn,       
            bucket_boundaries=limites, # Límites de los buckets
            bucket_batch_sizes=bucket_batch_sizes,  # Tamaños de batch para cada bucket
            padded_shapes=(
                [None, 48, 48],  
                [None]           
            ),
            padding_values=(
                tf.constant(0, dtype=tf.float32), # Relleno con ceros para los puntos 
                tf.constant(0, dtype=tf.int32) # Relleno con ceros para las traducciones 
            )
    )

# Aplicar bucketing y padding
train_dataset= train_dataset.apply(bucketed_dataset)
val_dataset= val_dataset.apply(bucketed_dataset)
test_dataset= test_dataset.apply(bucketed_dataset)



In [27]:
def count_batches(dataset):
    batch_count = 0
    for _ in dataset:
        batch_count += 1
    return batch_count
print(f"Cantidad de batches en train: {count_batches(train_dataset)}")
print(f"Cantidad de batches en val: {count_batches(val_dataset)}")
print(f"Cantidad de batches en test: {count_batches(test_dataset)}")


2024-10-08 14:02:07.955294: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Cantidad de batches en train: 24
Cantidad de batches en val: 10
Cantidad de batches en test: 10


2024-10-08 14:02:09.047866: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Función de pérdida

In [33]:
# def ctc_loss(translation, prediction):
    
#     input_length = tf.fill([tf.shape(prediction)[0], 1], tf.shape(prediction)[1])  # Longitud de la secuencia de entrada
#     label_length = tf.math.count_nonzero(translation, axis=-1, keepdims=True)   # Longitud de la secuencia de traducción
#     # Calcular la pérdida CTC usando ctc_batch_cost
#     loss = tf.keras.backend.ctc_batch_cost(translation, prediction, input_length, label_length)
#     return loss

**Buscar mejores parámetros**

In [ ]:
# import keras_tuner as kt
# num_classes=len(tokenizer.word_index) + 1

# # Definir el modelo para la búsqueda de hiperparámetros
# def build_model(hp):

#     def createCNNkt():

#         model= models.Sequential()
#         # Añadir capas CNN dinámicamente según el número de capas que se elija (1 a 5 capas CNN)
#         for i in range(hp.Int('num_cnn_layers', 2, 5)):  # De 1 a 5 capas CNN
#             model.add(layers.Conv2D(
#                 filters=hp.Int(f'conv_{i+1}_filters', min_value=64, max_value=256, step=32),
#                 kernel_size=(3, 3),
#                 activation='relu',
#                 input_shape=(48, 48, 1) if i == 0 else None))
#             model.add(layers.MaxPooling2D((2, 2)))

#         model.add(layers.Flatten())
#         return model

#     def createLSTMkt():

#         model= models.Sequential()
#         # Añadir capas LSTM dinámicamente según el número de capas que se elija (1 a 5 capas LSTM)
#         for i in range(hp.Int('num_lstm_layers', 2, 5)):  # De 1 a 5 capas LSTM
#             model.add(layers.LSTM(
#                 units=hp.Int(f'lstm_{i+1}_units', min_value=64, max_value=256, step=32),
#                 return_sequences=True))

#             # Añadir Dropout para cada capa LSTM
#             model.add(layers.Dropout(hp.Float(f'dropout_{i+1}', min_value=0.2, max_value=0.4, step=0.1)))
#         return model

#     cnn = createCNNkt()

#     video_input = layers.Input(shape=(None, 48, 48, 1))

#     # Aplicar CNN a cada frame usando TimeDistributed
#     cnn_features = layers.TimeDistributed(cnn)(video_input)

#     lstm=createLSTMkt()
#     lstm_out= lstm(cnn_features)
#     # Capa final de salida
#     output = layers.Dense(num_classes, activation='linear')(lstm_out)

#     # Compilar el modelo
#     model.compile(optimizer='adam', loss=ctc_loss)

#     model = models.Model(inputs=video_input, outputs=output)

#     return model

# # Definir la búsqueda de hiperparámetros con Keras Tuner
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_loss', # Métrica de evaluación
#     max_epochs= 50,
#     factor=3,
#     directory='',
#     project_name='cnn_lstm_hyperparam'
# )

In [ ]:
# # Realizar la búsqueda de hiperparámetros
# tuner.search(train_dataset, validation_data=val_dataset)

# best_model=tuner.get_best_model(1)[0]
# # Obtener los mejores hiperparámetros encontrados
# best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# print(best_hyperparams)

**Crear Modelo**

CNN

In [28]:
def create_cnn():
    model = models.Sequential()
    model.add(layers.Input(shape=(48, 48, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    return model

In [ ]:
cnn_example_model = create_cnn()


In [ ]:
cnn_example_model(points_batch)

LSTM

In [29]:
def create_lstm():
    model=models.Sequential()
    # Primera capa LSTM con return_sequences=True
    model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True)))
    model.add(layers.Dropout(0.3)) # Añadir Dropout
    
    # Segunda capa LSTM con return_sequences=True
    model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True)))
    model.add(layers.Dropout(0.3))

    return model

CNN+LSTM

In [30]:
def create_cnn_lstm_model(num_classes):
    cnn = create_cnn()
    
    video_input = layers.Input(shape=(None, 48, 48, 1)) 
    # masked_input = layers.Masking(mask_value=0.0)(video_input)

    
    # Aplicar CNN a cada frame usando TimeDistributed
    cnn_features = layers.TimeDistributed(cnn)(video_input)
    
    lstm=create_lstm()
    lstm_out= lstm(cnn_features)
    # Capa final de salida
    output = layers.Dense(num_classes, activation='linear')(lstm_out)
    
    model = models.Model(inputs=video_input, outputs=output)
    return model

Implementación para el entrenamiento

Usar bleu score

In [36]:
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 142.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 336.3 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [37]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(references, hypotheses):
    smoothing = SmoothingFunction().method4
    return np.mean([sentence_bleu([ref], hyp, smoothing_function=smoothing) for ref, hyp in zip(references, hypotheses)])

In [40]:
num_classes=len(tokenizer.word_index) + 1
model = create_cnn_lstm_model(num_classes)

optimizador = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizador)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/best_model.keras',
    save_best_only=True,       # Guardar solo si es el mejor modelo hasta ahora
    monitor='val_loss',
    mode='min'  
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)

In [45]:
ctc_loss=tf.keras.losses.CTC()

In [50]:
ctc_loss

In [46]:
def decode(predictions,greedy):
    predicted_classes = tf.argmax(predictions, axis=-1)
    mask = tf.not_equal(predicted_classes, 0)
    seq_lengths = tf.reduce_sum(tf.cast(mask, tf.int32), axis=-1)
    
    predictions=tf.transpose(predictions,[1,0,2])

    if greedy:
        decoded, _ = tf.nn.ctc_greedy_decoder(inputs=predictions, sequence_length=seq_lengths)
    
    else:
        decoded, _ = tf.nn.ctc_beam_search_decoder(inputs=predictions, sequence_length=seq_lengths)
    decoded_dense = tf.sparse.to_dense(decoded[0], default_value=-1).numpy()
    
    # Extraer las secuencias decodificadas eliminando los -1 (valores de relleno)
    decoded_sequences = []
    for seq in decoded_dense:
        decoded_sequences.append([val for val in seq if val != -1])
    
    return decoded_sequences

In [53]:
print(num_classes)

7664


In [60]:
def trainstep(model, points_batch, translation_batch,step, epoch_loss_avg, epoch_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(points_batch, training=True)
        print(tf.shape(predictions))
        print(tf.shape(translation_batch))

        loss = tf.reduce_mean(ctc_loss(translation_batch, predictions))
        
    # Calcular y aplicar gradientes
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # if step % 50 == 0:
    #     print(f"Step {step}: loss = {loss.numpy()}")
    predictions_decoded=greedy_decode(predictions, greedy=True)
    epoch_accuracy.update_state(translation_batch, predictions_decoded)
    epoch_loss_avg.update_state(loss)

In [48]:
def valstep(model, points_batch, translation_batch,val_loss_avg, val_acc_metric, val_bleu_avg):
    # Evaluar en el conjunto de validación al final de cada epoch usando BLEU
   
    predictions = model(points_batch, training=False)

    val_loss=tf.reduce_mean(ctc_loss(translation_batch,predictions))
    val_loss_avg.update_state(val_loss)

    val_predictions = decode(predictions, greedy=False)
    val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in val_predictions]


    val_acc_metric.update_state(translation_batch,val_predictions)
    val_references = [tokenizer.sequences_to_texts([ref.numpy()]) for ref in translation_batch]
    val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in np.argmax(val_predictions, axis=-1)]
    val_bleu = calculate_bleu(references = val_references, hypotheses = val_hypotheses)
    val_bleu_avg.update_state(val_bleu)
    return val_acc_metric.result(), val_loss_avg.result(), val_bleu_avg.result()

In [59]:
points_batch, translation_batch = next(iter(train_dataset_tqdm))
points_batch.shape, translation_batch.shape

(TensorShape([16, 488, 48, 48]), TensorShape([16, 373]))

In [61]:
from tqdm import tqdm
# Entrenar el modelo usando un ciclo de entrenamiento personalizado
epochs = 100
model.compile(optimizer=optimizador, loss=tf.keras.losses.CTC(), metrics=[calculate_bleu])

# Iterar a través de los batches y ajustar manualmente los parámetros
for epoch in range(epochs):
    
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    print(f"Epoch {epoch + 1}/{epochs}")
    
    train_dataset_tqdm = tqdm(train_dataset, desc="Entrenamiento", unit="batch")
    for step, (points_batch, translation_batch) in enumerate(train_dataset_tqdm):
        
        trainstep(model, points_batch, translation_batch, step, epoch_loss_avg, epoch_accuracy)
        
        train_dataset_tqdm.set_postfix({
            "Loss": epoch_loss_avg.result().numpy(),
            "Accuracy": epoch_accuracy.result().numpy()
        })
    
    # Métricas para la validación
    val_loss_avg = tf.keras.metrics.Mean()
    val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    val_bleu_avg = tf.keras.metrics.Mean()
    
    val_dataset_tqdm = tqdm(val_dataset, desc="Validación", unit="batch")
    for step, (points_batch, translation_batch) in enumerate(val_dataset_tqdm):
        
        val_acc, val_loss, val_bleu = valstep(model, points_batch, translation_batch,val_loss_avg, val_acc_metric, val_bleu_avg)
        
        val_dataset_tqdm.set_postfix({
            "Val Loss": val_loss.numpy(),
            "Val Accuracy": val_acc.numpy(),
            "Val BLEU": val_bleu
        })

    checkpoint_callback.on_epoch_end(epoch, logs={'val_loss': val_loss, 'val_accuracy': val_acc, 'val_bleu': val_bleu})

    if early_stopping.on_epoch_end(epoch, logs={'val_loss': val_loss, 'val_accuracy': val_acc, 'val_bleu': val_bleu}):
        print("Early stopping triggered")
        break
    print(f"Epoch: {epoch}, Loss: {epoch_loss_avg}, Accuracy: {epoch_accuracy}")
    print(f'Epoch: {epoch}, val_loss: {val_loss_avg}, val_acc: {val_acc_metric}, val_bleu: {val_bleu_avg}')

# Evaluar el modelo en el dataset de test
#test_loss, test_acc = model.evaluate(test_dataset)
#print(f'Test accuracy: {test_acc}')

Epoch 1/100


Entrenamiento: 0batch [00:00, ?batch/s]

Entrenamiento: 0batch [03:37, ?batch/s]

tf.Tensor([  16 7664], shape=(2,), dtype=int32)
tf.Tensor([ 16 373], shape=(2,), dtype=int32)


ValueError: Logits `y_pred` are expected to be a tensor of shape `(batch_size, max_length, num_classes)`. Received: y_pred.shape=(16, 7664)